In [8]:
import pandas as pd
import eform_query_processor as eqp
import os

In [16]:
# setting up the absolute savepath
save_path = r'C:\Users\aange\OneDrive\Desktop\Personal Python Projects\ferc_data_project\ferc_714_xml_files_test'

save_path_new = r'C:\Users\AngelSalazar\python_scratch\ferc_data_project\ferc_714_xml_files_test'

In [10]:
# loading in df containing files we want to grab
source_df = pd.read_csv('process_check/eform_filtered_data_2024-09-14.csv')

source_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   CID        153 non-null    object
 1   Company    153 non-null    object
 2   Form       153 non-null    object
 3   Year       153 non-null    int64 
 4   Period     153 non-null    object
 5   Date/Time  153 non-null    object
 6   Status     153 non-null    object
 7   Filing ID  153 non-null    int64 
dtypes: int64(2), object(6)
memory usage: 9.7+ KB


In [11]:
# checking the first 5
source_df_f5 = source_df[:5]

print(source_df_f5.info())

print(source_df_f5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   CID        5 non-null      object
 1   Company    5 non-null      object
 2   Form       5 non-null      object
 3   Year       5 non-null      int64 
 4   Period     5 non-null      object
 5   Date/Time  5 non-null      object
 6   Status     5 non-null      object
 7   Filing ID  5 non-null      int64 
dtypes: int64(2), object(6)
memory usage: 452.0+ bytes
None
       CID                                            Company      Form  Year  \
0  C000771                         Southwest Power Pool, Inc.  Form 714  2023   
1  C000771                         Southwest Power Pool, Inc.  Form 714  2023   
2  C000771                         Southwest Power Pool, Inc.  Form 714  2023   
3  C004245  Salt River Project Agricultural Improvement an...  Form 714  2023   
4  C000771                         Southwest 

In [12]:
# create a list of filing id
filing_ids = list(source_df_f5['Filing ID'])
print(filing_ids)

[192310, 192309, 192306, 192277, 182167]


In [39]:
# return files
eqp.return_eform_filings(filing_ids=filing_ids, 
                         save_path=save_path_new)

Southwest Power Pool, Inc._ferc_714_Q42023_192310.xml
Southwest Power Pool, Inc._ferc_714_Q42023_192309.xml
Southwest Power Pool, Inc._ferc_714_Q42023_192306.xml
Salt River Project Agricultural Improvement and Power District_ferc_714_Q42023_192277.xml
Southwest Power Pool, Inc._ferc_714_Q42023_182167.xml


Now we want to get a sense of how to sort our directories by time stamp
so that we can return a list of file paths to match with out pre existing
filing ids dataframe, allowing us to rename duplicate named files to assess
which files may have been revises or simply reuploaded at a later time.

In [18]:
# function to sort directory provided by either name or time
def sort_directory(sort_by: str, dir_path: str):

    '''
    In essence, first select whether you want to sort directory by name or by date created
    '''

    # purpose is to return the time the file was created
    def get_creation_time(item):
        item_path = os.path.join(dir_path, item)
        return os.path.getctime(item_path)
    
    # items meaning the files wihthin the directory path defined
    items = os.listdir(dir_path)

    if sort_by == 'date':
        '''
        in this case, user would like to sort directory path files by date
        '''
        print('Sorting files in path by creation date!')

        # sorting our items from oldest to newest
        sorted_items = sorted(items, key=get_creation_time)

        return sorted_items

    elif sort_by == 'name':
        '''
        in this case we would like to sort directory path files by name
        '''
        print('Sorting files in path by name!')

    else: 
        print('Please redefine how you would like to sort!')


In [19]:
# folder_path to test
fpath_714 = 'ferc_714_xml_files_test'

# attempt to check function
test_case = sort_directory(sort_by='date', dir_path=fpath_714)
print(test_case)


Sorting files in path by creation date!
['spp-20231231.xbrl', 'spp-20231231 (1).xbrl', 'spp-20231231 (2).xbrl', 'wk-20231231.xml', 'spp-20231231 (3).xbrl']


In [20]:
# function to change date times to the following format so that we can use it to rename files
# 12/9/2024, 4:25 PM --> 12_9_2024_4_25_PM 
# replace any [/], [,], [space] and [:] with an undescore

def clean_subnames(subname_list: list):
    '''
    given a string of a date or cname convert any defined chars to underscores
    '''
    # edited subname list
    new_subname_list = []

    for subname in subname_list:
        # first iteration checks for /
        iteration_1 = subname.replace("/", "_")
        iteration_2 = iteration_1.replace(",", "_")
        iteration_3 = iteration_2.replace(" ", "_")
        iteration_4 = iteration_3.replace(".", "_")
        final_iteration = iteration_4.replace(":", "_")
        new_subname_list.append(final_iteration)

    return new_subname_list


test_date_rename = clean_subnames(subname_list=["7/9/2024, 4:25 PM"])
cname_rename = clean_subnames(subname_list=['Southwest Power Pool, Inc.'])
print(test_date_rename)
print(cname_rename)

['7_9_2024__4_25_PM']
['Southwest_Power_Pool__Inc_']


In [21]:
# function aimed at setting up the renaming of files by grabbing info from df we used to grab files
def rename_xml_base(dframe: object):
    '''
    in essence, go through df and set up framework for the renaming of xml files grabbed
    '''
    # a finalized list of the renamed xml names
    renamed_xmls = []

    # columns to combine and combined strings list
    columns_to_combine = ['Form', 'Year', 'Company', 'Period', 'Date/Time', 'Filing ID']

    # combined strings
    for index, row in dframe.iterrows():

        # for each row in our dataframe we are combining select col values
        combined_string = ' '.join(row[columns_to_combine].astype(str))

        # in essence now changing all sub text into underscores
        final_string = clean_subnames(subname_list=list(combined_string))

        # combined final string where we have converted all sub elements in our words
        f_string = ''.join([str(item) for item in final_string])

        # printing combined string to see what's up
        renamed_xmls.append(f_string)
    
    return renamed_xmls



In [22]:
x = rename_xml_base(dframe=source_df_f5)
print(x)


['Form_714_2023_Southwest_Power_Pool__Inc__Q4_7_9_2024__4_25_PM_192310', 'Form_714_2023_Southwest_Power_Pool__Inc__Q4_7_9_2024__1_58_PM_192309', 'Form_714_2023_Southwest_Power_Pool__Inc__Q4_7_8_2024__9_13_AM_192306', 'Form_714_2023_Salt_River_Project_Agricultural_Improvement_and_Power_District_Q4_7_3_2024__12_24_PM_192277', 'Form_714_2023_Southwest_Power_Pool__Inc__Q4_6_17_2024__8_45_AM_182167']


okay, so now we have the tools to rename a file with a unique name that 
gives us information on who filed and exactly when, 

this makes it easy to parse through files that have been submitted multiple times
by specific entities....


now we want to work on 
(1) sorting the files within a directory
(2) iterating throuhg this file directory and renaming files with the processed
    file xml namees

we want to make sure we are accurately naming so we don't incorrecly name any files

In [35]:
# now, iterate through each file and rename it with what is in the list...
def rename_all_files(xml_df: object, dir_path: str, sort_by = 'date'):

    '''
    now we will first iterate through xml df, grab rows of data to set up a unique file name
    and then rename the file to that name

    xml_df = df we used to pull the files from the ferc website
    sort_by = set to 'date' default but can also be set to 'name'
    dir_path = path of where the files downloaded is at....
    '''
    print('here is what we are working with: ')
    print(f'inputted df: {xml_df}')
    print(f'dir_path: {dir_path}')
    print('\n')

    # first, we are defining the directory path where we will be making the changes
    files_list = sort_directory(sort_by=sort_by, dir_path=dir_path)
    print(f'files list: {files_list}')

    # renaming xml files
    temp_xmls = rename_xml_base(dframe=xml_df)
    print(f'temp_xmls: {temp_xmls}')  

    # testing to see if things are matched 
    for file in files_list:
        print(f'{file}, {temp_xmls[files_list.index(str(file))]}')


In [37]:
# PLACEHOLDER
# now, iterate through each file and rename it with what is in the list...
def rename_all_files(xml_df: object, dir_path: str, sort_by = 'date'):

    '''
    now we will first iterate through xml df, grab rows of data to set up a unique file name
    and then rename the file to that name

    xml_df = df we used to pull the files from the ferc website
    sort_by = set to 'date' default but can also be set to 'name'
    dir_path = path of where the files downloaded is at....
    '''
    print('here is what we are working with: ')
    print(f'inputted df: {xml_df}')
    print(f'dir_path: {dir_path}')
    print('\n')

    # first, we are defining the directory path where we will be making the changes
    files_list = sort_directory(sort_by=sort_by, dir_path=dir_path)
    print(files_list)

    # renaming xml files
    temp_xmls = rename_xml_base(dframe=xml_df)
    print(temp_xmls)  

    # now we want to iterate through each file and rename it
    for file in files_list: 
        # true file path
        true_file_path = f'{dir_path}/{file}'
        print(true_file_path)

        # new name
        print(f'new file name: {temp_xmls[file.index(str(file))]}')

        # rename file by grabbing location of the 'file' element
        new_fpath = os.rename(f'{true_file_path}', 
                              f'{dir_path}/{temp_xmls[files_list.index(str(file))]}.xml')




it looks like we need to split things between 
1. folder path
2. old file path
3. new file path

In [40]:
# folder_path to test
folder_path = 'ferc_714_xml_files_test'

rename_all_files(xml_df=source_df_f5, dir_path=fpath_714)


here is what we are working with: 
inputted df:        CID                                            Company      Form  Year  \
0  C000771                         Southwest Power Pool, Inc.  Form 714  2023   
1  C000771                         Southwest Power Pool, Inc.  Form 714  2023   
2  C000771                         Southwest Power Pool, Inc.  Form 714  2023   
3  C004245  Salt River Project Agricultural Improvement an...  Form 714  2023   
4  C000771                         Southwest Power Pool, Inc.  Form 714  2023   

  Period           Date/Time    Status  Filing ID  
0     Q4   7/9/2024, 4:25 PM  Accepted     192310  
1     Q4   7/9/2024, 1:58 PM  Accepted     192309  
2     Q4   7/8/2024, 9:13 AM  Accepted     192306  
3     Q4  7/3/2024, 12:24 PM  Accepted     192277  
4     Q4  6/17/2024, 8:45 AM  Accepted     182167  
dir_path: ferc_714_xml_files_test


Sorting files in path by creation date!
['spp-20231231.xbrl', 'spp-20231231 (1).xbrl', 'spp-20231231 (2).xbrl', 'wk-2